In [2]:
from truckscenes import TruckScenes   
version = 'v1.0-mini'
dataroot = '/data/mint/Radar_Dataset/ManTruck/man-truckscenes/'
trucksc = TruckScenes(version=version, dataroot=dataroot, verbose=True)

Loading truckscenes tables for version v1.0-mini...
11 attribute,
18 calibrated_sensor,
27 category,
20090 ego_motion_cabin,
20089 ego_motion_chassis,
20116 ego_pose,
1094 instance,
400 sample,
25750 sample_annotation,
43556 sample_data,
10 scene,
18 sensor,
4 visibility,
Done loading in 0.835 seconds.
Reverse indexing ...
Done reverse indexing in 0.2 seconds.
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [14]:
import os
from truckscenes.utils.data_classes import LidarPointCloud, RadarPointCloud

def get_all_frames(scene):
    print("=" * 100)
    print("[#] scene:", scene)
    print(scene.keys())
    count = 0
    samples_token = [scene['first_sample_token']]
    print("[#] First_sample_token: ", scene['first_sample_token'])
    print("[#] Last_sample_token: ", scene['last_sample_token'])
    start_sample = trucksc.get('sample', samples_token[0])
    current_sample = start_sample
    assert start_sample['prev'] == '', "[#] First sample should not have a previous sample."

    while current_sample['next'] != '':
        # print("[#] Sample:", current_sample['token'])
        count += 1
        current_sample = trucksc.get('sample', current_sample['next'])
        samples_token.append(current_sample['token'])
        # print("[#] Next sample:", current_sample['token'])
        assert current_sample['scene_token'] == scene['token'], "[#] Sample should belong to the same scene."
    
    samples_token.append(scene['last_sample_token'])
    count += 1
    assert trucksc.get('sample', samples_token[-1])['next'] == '', "[#] Last sample should not have a next sample."
    print("[#] #count:", count)
    print("[#] #nbr_samples: ", scene['nbr_samples'])
    assert count == scene['nbr_samples'], "[#] Count should be equal to the number of samples in the scene."
    print("=" * 100)

    return {scene['token']: samples_token}
    
print("[#] #N-scenes:", len(trucksc.scene))
samples_token = get_all_frames(trucksc.scene[0])
for k, v in samples_token.items():
    print("[#] Scene token:", k)
    print("[#] #samples_token:", len(v))
    print("[#] First sample token:", v[0])
    print("[#] Last sample token:", v[-1])
    print("=" * 100)
    for i in range(len(v)):
        sample = trucksc.get('sample', v[i])
        if i == 0:
            print("[#] Example of the first sample:")
            print(sample.keys())
            print(sample['data'].keys())
            radar = trucksc.get('sample_data', sample['data']['RADAR_RIGHT_FRONT'])
            pcl_path = os.path.join(trucksc.dataroot, radar['filename'])
            radar_dat = RadarPointCloud.from_file(pcl_path)
            radar_pc = radar_dat.points.transpose(1, 0)
            print("[#] Radar data shape:", radar_pc.shape)
        # print("=" * 100)

[#] #N-scenes: 10
[#] scene: {'name': 'scene-0044384af3d8494e913fb8b14915239e-3', 'description': 'weather.clear;area.terminal;daytime.noon;season.autumn;lighting.illuminated;structure.regular;construction.unchanged', 'token': '044c648ac12345f1aedf33c9f91cdc5a', 'log_token': '', 'nbr_samples': 40, 'first_sample_token': '32d2bcf46e734dffb14fe2e0a823d059', 'last_sample_token': '94640f753b284a1c9c0e8694243f60cd'}
dict_keys(['name', 'description', 'token', 'log_token', 'nbr_samples', 'first_sample_token', 'last_sample_token'])
[#] First_sample_token:  32d2bcf46e734dffb14fe2e0a823d059
[#] Last_sample_token:  94640f753b284a1c9c0e8694243f60cd
[#] #count: 40
[#] #nbr_samples:  40
[#] Scene token: 044c648ac12345f1aedf33c9f91cdc5a
[#] #samples_token: 41
[#] First sample token: 32d2bcf46e734dffb14fe2e0a823d059
[#] Last sample token: 94640f753b284a1c9c0e8694243f60cd
[#] Example of the first sample:
dict_keys(['token', 'scene_token', 'timestamp', 'prev', 'next', 'data', 'anns'])
dict_keys(['RADAR_RI